# ReasonForge

Deterministic math & code tools for small language models.

1. Clone repo & install deps
2. Install Ollama & pull model
3. Sanity tests
4. MATH-500 / HumanEval benchmarks
5. Gradio chat UI

---
## Setup

In [ ]:
# Clone repo & install deps
!git clone https://github.com/RoyCoding8/MCP.git /content/MCP 2>/dev/null || echo 'Already cloned'
!pip install -q "requests>=2.31.0" "gradio>=6.0" "sympy>=1.13.0" "datasets>=4.6.1" "math-verify[antlr4_13_2]>=0.9.0"

In [ ]:
# Install Ollama
!sudo apt-get install -qq zstd
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# Set USE_DRIVE=True to cache models on Google Drive across sessions
import os
USE_DRIVE = True

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    MODELS_PATH = '/content/drive/MyDrive/ollama_models'
else:
    MODELS_PATH = '/content/ollama_models'

os.makedirs(MODELS_PATH, exist_ok=True)
print(f'Models: {MODELS_PATH}')

In [ ]:
import subprocess, time, threading, requests

subprocess.run(['pkill', '-f', 'ollama'], capture_output=True)
time.sleep(2)

env = os.environ.copy()
env['OLLAMA_MODELS'] = MODELS_PATH
env['OLLAMA_HOST'] = '0.0.0.0:11434'

log_file = open('/content/ollama_server.log', 'w')
proc = subprocess.Popen(
    ['ollama', 'serve'], env=env,
    stdout=log_file, stderr=subprocess.STDOUT,
)

def tail_log(path, stop):
    with open(path, 'r') as f:
        while not stop.is_set():
            line = f.readline()
            if line: print(line, end='', flush=True)
            else: time.sleep(0.3)

_stop_tail = threading.Event()
threading.Thread(target=tail_log, args=('/content/ollama_server.log', _stop_tail), daemon=True).start()

for i in range(30):
    try:
        requests.get('http://localhost:11434/', timeout=2)
        print(f'\nOllama ready ({i+1}s)')
        break
    except requests.exceptions.RequestException:
        time.sleep(1)
else:
    print('\nOllama failed to start — check logs above')

In [ ]:
#Pull Model
MODEL = 'qwen3:32b'

print(f'Pulling {MODEL}...')
!OLLAMA_MODELS={MODELS_PATH} ollama pull {MODEL}

---
## Verify

In [ ]:
os.chdir('/content/MCP')
!python -m tests.sanity

---
## Benchmarks

A/B comparison: Baseline (no tools) vs ReasonForge (with tools).

In [ ]:
# MATH-500 Benchmark
N_MATH = 50
SKIP_BASELINE = False
THINK = True

os.chdir('/content/MCP')
cmd = f'python -m tests.benchmark --model {MODEL} --n {N_MATH}'
if SKIP_BASELINE: cmd += ' --skip-baseline'
if THINK: cmd += ' --think'
print(f'Running: {cmd}\n')
!{cmd}

In [ ]:
# HumanEval Code Benchmark
N_CODE = 20
SKIP_BASELINE_CODE = False
THINK_CODE = True

os.chdir('/content/MCP')
cmd = f'python -m tests.code_benchmark --model {MODEL} --n {N_CODE}'
if SKIP_BASELINE_CODE: cmd += ' --skip-baseline'
if THINK_CODE: cmd += ' --think'
print(f'Running: {cmd}\n')
!{cmd}

---

In [ ]:
os.environ['RF_SHARE'] = '1'
os.chdir('/content/MCP')
!python -m ui.app

---

In [ ]:
# !tail -50 /content/ollama_server.log

In [ ]:
# !OLLAMA_MODELS=$MODELS_PATH ollama ps
# print()
# !OLLAMA_MODELS=$MODELS_PATH ollama list

In [ ]:
# !rm -rf /content/MCP
# !rm /content/ollama_server.log